# How it works

**AnnCollectionDataset** directly reads h5ad files through [AnnCollection](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/anncollection.html) and serve data using [LitData](https://github.com/Lightning-AI/litdata) frontend. To use dataset, we first need to prepare dataset index folder that has multiple splits.

# Building LitData index

In [1]:
from pathlib import Path


In [2]:
src_root = Path("/dccstor/bmfm-targets/data/omics/transcriptome/scRNA/finetune")#("/proj/bmfm/usr/data")#
src_root

PosixPath('/dccstor/bmfm-targets/data/omics/transcriptome/scRNA/finetune')

In [3]:
dst_root = src_root / "AnnCollection"
dst_root

PosixPath('/dccstor/bmfm-targets/data/omics/transcriptome/scRNA/finetune/AnnCollection')

# Prepare dataset

In [4]:
from bmfm_targets import config
from bmfm_targets.config import LabelColumnInfo
from bmfm_targets.datasets.anncollection import AnnCollectionDataModule
from bmfm_targets.tokenization import get_gene2vec_tokenizer

/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/dccstor/bmfm-targets/users/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/pandera/engines/pandas_engine.py:67: UserWarning: Using typeguard < 3. Generic types like List[TYPE], Dict[TYPE, TYPE] will only validate the first element in the collection.
  warnings.warn(


In [21]:
from importlib import reload

import bmfm_targets.datasets.anncollection

reload(bmfm_targets.datasets.anncollection)

<module 'bmfm_targets.datasets.anncollection' from '/dccstor/bmfm-targets/users/a1kosugi/gsetests/bmfm/bmfm_targets/datasets/anncollection/__init__.py'>

In [22]:
from importlib import reload

import bmfm_targets.datasets.anncollection.anncollection_dataset

reload(bmfm_targets.datasets.anncollection.anncollection_dataset)


<module 'bmfm_targets.datasets.anncollection.anncollection_dataset' from '/dccstor/bmfm-targets/users/a1kosugi/gsetests/bmfm/bmfm_targets/datasets/anncollection/anncollection_dataset.py'>

### Prepare index for 3 sets of data sources.

In [5]:
def gene2vec_fields():
    gene2vec_field_dicts = [
        {
            "field_name": "genes",
            "pretrained_embedding": None,
            "is_masked": False,
            "vocab_update_strategy": "static",
        },
        {
            "field_name": "expressions",
            "pretrained_embedding": None,
            "is_masked": True,
            "vocab_update_strategy": "static",
        },
    ]

    gene2vec_fields = [config.FieldInfo(**fd) for fd in gene2vec_field_dicts]
    tokenizer = get_gene2vec_tokenizer()
    for field in gene2vec_fields:
        field.update_vocab_size(tokenizer)
    return gene2vec_fields

In [6]:
params ={
    "test_scp259changepi": {
        "src": [src_root / "scIBD300K" / "processed_42.h5ad", src_root / "ChangEpi" / "processed_42.h5ad"],
        "labels":["celltype", "Final_CellType"]
    },
    "test_scp259changall": {
        "src": [src_root / "scIBD300K" / "processed_42.h5ad", src_root / "ChangAll" / "processed_42.h5ad"],
        "labels":["celltype", "Final_CellType"]
    },
    "test_scp259changall_gp": {
        "src": [src_root / "scIBD300K" / "processed_42.h5ad", src_root / "ChangAll" / "processed_Group_CellType_42.h5ad"],
        "labels":["celltype", "Group_CellType"]
    },
}
# "Celltype" for scp1884

In [7]:
for key in params:
    print(f"preparing for {key}")
    label_columns = [LabelColumnInfo(label_column_name=label, is_stratification_label=True) for label in params[key]["labels"]]
    dataset_kwargs = {
        "dataset_paths": params[key]["src"],
        #"dataset_dir": dst_root / key / "data",
        "index_dir": dst_root / key / "index",
    }
    #transform_kwargs = {
    #    "stratifying_labels": params[key]["labels"],
    #}
    tokenizer = get_gene2vec_tokenizer()
    pars = {
        "tokenizer": tokenizer,
        "batch_size": 2,
        "fields": gene2vec_fields(),
        "num_workers": 8,
        "mlm": False,
        "shuffle": True,
        "sequence_order": "sorted",
        "collation_strategy": "sequence_classification",
        "label_columns": label_columns,
        "dataset_kwargs": dataset_kwargs,
        "transform_datasets":True,
        #"transform_kwargs": transform_kwargs,
    }
    datamodule = AnnCollectionDataModule(**pars)
    datamodule.prepare_data()
    datamodule.setup("fit")

preparing for test_scp259changepi


/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is d

preparing for test_scp259changall


/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is d

preparing for test_scp259changall_gp


/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is d

### Parameters that normally have to be set in yaml file (see PanglaoDB yaml files).

In [ ]:
dataset_kwargs = {
    "dataset_dir": dst_root / "changall_scp1884_scp259" / "ALL",
    "index_dir": dst_root / "changall_scp1884_scp259" / "litdata_index",
    "label_columns": ["Celltype", "celltype", "Final_CellType"],
}
tokenizer = get_gene2vec_tokenizer()
pars = {
    "tokenizer": tokenizer,
    "batch_size": 2,
    "fields": gene2vec_fields(),
    "num_workers": 8,
    "mlm": False,
    "collation_strategy": "sequence_classification",
    "dataset_kwargs": dataset_kwargs,
}

In [12]:
datamodule = AnnCollectionDataModule(**pars)
datamodule.prepare_data()
datamodule.setup("fit")


/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [13]:
datamodule.label_dict

{'Celltype': {'Activated fibroblasts CCL19 ADAMADEC1': 0,
  'B cells': 1,
  'B cells AICDA LRMP': 2,
  'Cycling cells': 3,
  'DC1': 4,
  'DC2 CD1D': 5,
  'DC2 CD1D-': 6,
  'Endothelial cells CA4 CD36': 7,
  'Endothelial cells CD36': 8,
  'Endothelial cells DARC': 9,
  'Endothelial cells LTC4S SEMA3G': 10,
  'Enterochromaffin cells': 11,
  'Enterocytes BEST4': 12,
  'Enterocytes CA1 CA2 CA4-': 13,
  'Enterocytes TMIGD1 MEP1A': 14,
  'Enterocytes TMIGD1 MEP1A GSTA1': 15,
  'Enteroendocrine cells': 16,
  'Epithelial Cycling cells': 17,
  'Epithelial HBB HBA': 18,
  'Epithelial cells METTL12 MAFB': 19,
  'Fibroblasts ADAMDEC1': 20,
  'Fibroblasts KCNN3 LY6H': 21,
  'Fibroblasts NPY SLITRK6': 22,
  'Fibroblasts SFRP2 SLPI': 23,
  'Fibroblasts SMOC2 PTGIS': 24,
  'Glial cells': 25,
  'Goblet cells MUC2 TFF1': 26,
  'Goblet cells MUC2 TFF1-': 27,
  'Goblet cells SPINK4': 28,
  'IELs ID3 ENTPD1': 29,
  'ILCs': 30,
  'Immune Cycling cells': 31,
  'Inflammatory fibroblasts IL11 CHI3L1': 32,
  'L

## Look at a sample in dataset

In [14]:
dataset=datamodule.train_dataset

In [15]:
ds = dataset[1]
ds.metadata

{'Celltype': nan,
 'celltype': nan,
 'Final_CellType': 'TA_2',
 'cell_name': 'HA01_AC_AAACCCAAGTCGGGAT.1'}

In [16]:
dataset[1000].metadata

{'Celltype': nan,
 'celltype': nan,
 'Final_CellType': 'EC_1',
 'cell_name': 'HA01_AC_CTTCTCTGTATACCCA.1'}

## Look at a collation result

In [17]:
train_dataloader = datamodule.train_dataloader()
itr = iter(train_dataloader)

In [18]:
for i in range(2):
    item = next(itr)
    c1 = dataset[i].metadata["cell_name"]
    c2 = item["cell_names"][0]
    print(f"{c1 == c2}: {c1} - {c2}")

True: HA01_AC_AAACCCAAGGCTGGAT.1 - HA01_AC_AAACCCAAGGCTGGAT.1
False: HA01_AC_AAACCCAAGTCGGGAT.1 - N104689_N2-CGTTGGGCATTGCGGC


In [19]:
print(f"{c1 == c2}: {c1} - {c2}")

False: HA01_AC_AAACCCAAGTCGGGAT.1 - N104689_N2-CGTTGGGCATTGCGGC


In [ ]:
train_dataloader = datamodule.train_dataloader()
item = next(iter(train_dataloader))
print(item)

thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/rayon-core-1.12.1/src/registry.rs:168:10:
The global thread pool has not been initialized.: ThreadPoolBuildError { kind: IOError(Os { code: 11, kind: WouldBlock, message: "Resource temporarily unavailable" }) }
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
Resource temporarily unavailable (src/thread.cpp:241)
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/rayon-core-1.12.1/src/registry.rs:168:10:
The global thread pool has not been initialized.: ThreadPoolBuildError { kind: IOError(Os { code: 11, kind: WouldBlock, message: "Resource temporarily unavailable" }) }
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
Resource temporarily unavailable (src/thread.cpp:241)
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/rayon-core-1.12.1/src/registry.rs:168:10:
Th

RuntimeError: can't start new thread

Traceback (most recent call last):
  File "/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
    se

: 